<img align="right" width="250" height="150" src="https://lh3.googleusercontent.com/p/AF1QipPWZQfa087JiVjutpUTVEGRnh6W214Wjm439gKQ=w1080-h608-p-no-v0">

## **Proyecto:** Retención de usuarios de la Plataforma Digital

#### El objetivo de este notebook es incorporar como datos agregados al archivo de Crosseling, los datos de Usuarios, Movimientos y Reclamos

## Importación de librerias y datos globales

In [241]:
import pandas as pd
import datetime as dt
import numpy as np
import os

LOCAL = True
if LOCAL:
    dir = os.path.abspath(r'..\Data\Export') + '\\'
else:
    # Monto la unidad de drive porque los archivos son muy grandes
    from google.colab import drive
    drive.mount('/content/drive')
    dir = '/content/drive/MyDrive/Trayecto3-DataDriven/Data/'

## Lectura de archivo de usuarios

In [231]:
file = dir + 'Usuarios.csv'
df_usuarios = pd.read_csv(file, encoding='latin-1', parse_dates=["FechaCreacion", "LAST_LOGIN"])

In [232]:
# Filtro los registros que se crearon hace menos de tres meses ya que todos ellos no se pueden evaluar
df_usuarios = df_usuarios[df_usuarios['FechaCreacion'] < dt.datetime(2022, 2, 1)]

In [233]:
fecha_corte =  dt.datetime(2022, 4, 30)
df_usuarios[df_usuarios['LAST_LOGIN'] > fecha_corte] = fecha_corte

In [234]:
# Calculo la cantidad de dias desde su último acceso a la plataforma 
df_usuarios['DIAS_SIN_USAR_PD'] = (fecha_corte - df_usuarios['LAST_LOGIN']).astype('timedelta64[D]')

In [235]:

# Calculo Antiguedad en dias la plataforma
df_usuarios['ANTIGUEDAD_PD'] = (df_usuarios['LAST_LOGIN'] - df_usuarios['FechaCreacion']).astype('timedelta64[D]')
# Filtro los registros sin antiguedad en la plataforma
df_usuarios = df_usuarios.loc[df_usuarios['ANTIGUEDAD_PD'] > 0]

In [236]:
# Asumo que abandonó la PD si lleva mas de 90 dias sin hacer login
df_usuarios["ABANDONO_PD"] = ['Sí' if ( _ > 90 ) else 'No' for _ in df_usuarios["DIAS_SIN_USAR_PD"]]

In [237]:
df_usuarios.sample(5)

,ID_USER,FechaCreacion,LAST_LOGIN,SegFactor,DIAS_SIN_USAR_PD,ANTIGUEDAD_PD,ABANDONO_PD
93689,43241bd2c0234b18ab24563eecb873cc,2020-12-28,2021-03-29,SMS,397.0,91.0,Sí
204662,978c21070c694fa09388c4c47e16b638,2019-06-03,2022-04-14,SMS,16.0,1046.0,No
163275,8ec1c6bf3d934ff3b3d464ae03d8f6c5,2020-07-25,2022-01-12,SMS,108.0,536.0,Sí
88700,541ec181d4614eb1a6d43588e1a8014a,2020-05-27,2021-03-03,SMS,423.0,280.0,Sí
38346,6acf8f1670fc459b9b7b3fdd0adec20b,2020-05-11,2020-08-04,SMS,634.0,85.0,Sí


## Lectura de archivo de Crosseling

Se lee el archivo de crosseling, se filtra para individuos y se cruza con el archivo de usuarios

In [238]:
def tratar_crosseling(periodo: str, df_usr: pd.DataFrame):

  tcols = {
    'PERIODO': str,
    'ID_USER': str,
    'RELACION': str,
    'TIPOIMPN': str,
    'NATUJURI': str,
    'SUCURSAL': str,
    'BCRASECT': str,
    'BCRASEGO': str,
    'GRUPO_CLIENTE': str,
    'GRUPOGENERAL': str,
    'SUBGRUPO': str,
    'SEGMENTO_SUELDOS': str,
    'SEGMENTO_BANCAS': str,
    'SALDO_COMPUTO_SIST_2': float,
    'SALDO_COMPUTO_SIST_CC_MB': float,
    'SALDO_COMPUTO_SIST_CA_P': float,
    'SALDO_COMPUTO_SIST_CC_P': float,
    'SALDO_COMPUTO_SIST_FCI': float,
    'FUM': str
  }

  file = dir + 'Crosseling_{}.csv'.format(periodo)
  df_crosseling = pd.read_csv(file, dtype=tcols, parse_dates=['FUM'], index_col=False)

  # Solo me quedo con los 'Individuo sin actividad comercial'
  df_crosseling = df_crosseling.loc[df_crosseling['GRUPOGENERAL'] == 'Individuo sin actividad comercial']
  df_crosseling.drop('GRUPOGENERAL', axis=1, inplace=True)

  # Cruzar con el archivo de usuarios y retornar
  return pd.merge(df_usuarios, df_crosseling, how='inner', on = 'ID_USER')

## Archivo de movimientos

El objeto de leer el archivo de movimientos es incorporar a la tabla de crosseling nuevas columnas con información de:

- Cantidad de movimientos monetarios y monto total realizados por medio de la plataforma digital
- Cantidad de movimientos monetarios y monto total realizados por FUERA de la plataforma digital
- Cantidad de movimientos NO monetarios y realizados por medio de la plataforma digital
- Cantidad de movimientos NO monetarios y realizados por FUERA de la plataforma digital

Total **6** nuevas variables

La idea es tener es que el algoritmo de predicción elegido tenga información adicional para poder inferir si el cliente abandona o no la plataforma

In [239]:
def tratar_movimientos(periodo: str, df_cons: pd.DataFrame):

    uc = [
        'ID_USER',
        'CANAL_RATIO',
        'CANT_OPERACIONES',
        'MONTO_TOTAL'
    ]

    tc = {
        'ID_USER': str,
        'CANAL_RATIO': str,
        'CANT_OPERACIONES': int,
        'MONTO_TOTAL': float,
    }

    df_estudio = df_cons
    
    #### Transacciones Monetarias ####

    file = dir + r'Sum_Trx_Monetarias_{}.csv'.format(periodo)
    df_tx = pd.read_csv(file, index_col=False, dtype=tc, usecols=uc)

    # Agrupo y sumarizo los registros por canales digitales y no digitales
    df_tx = df_tx.groupby([
        'ID_USER',
        'CANAL_RATIO']) \
        .aggregate('sum') \
        .sort_values(by='ID_USER') \
        .reset_index()

    # Pongo en columnas la cantidad y monto de las transacciones por canal digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']=='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES', 'MONTO_TOTAL']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_MON_DIG', 'MONTO_TOTAL': 'MONT_OP_MON_DIG'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    # Pongo en columnas la cantidad y monto de las transacciones por canal NO digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']!='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES', 'MONTO_TOTAL']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_MON_OTR', 'MONTO_TOTAL': 'MONT_OP_MON_OTR'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    #### Transacciones No Monetarias ####
    file = dir + r'Sum_Trx_NoMonetarias_{}.csv'.format(periodo)
    df_tx = pd.read_csv(file, index_col=False, dtype=tc, usecols=uc)

    # Agrupo los registros por canales digitales y no digitales
    df_tx = df_tx.groupby([
        'ID_USER',
        'CANAL_RATIO']) \
        .aggregate('sum') \
        .sort_values(by='ID_USER') \
        .reset_index()
        
    # Pongo en columnas la cantidad y monto de las transacciones por canal digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']=='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_NOMON_DIG'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    # Pongo en columnas la cantidad y monto de las transacciones por canal NO digital
    df_tx_join = df_tx \
        .loc[(df_tx['CANAL_RATIO']!='DIGITAL'), ['ID_USER', 'CANT_OPERACIONES']] \
        .rename(columns = {'CANT_OPERACIONES': 'CANT_OP_NOMON_OTR'})
    df_estudio = pd.merge(df_estudio, df_tx_join, how='left', on = 'ID_USER', indicator=False)

    # Convertir las columnas de cantidades a enteros previo sacar los valores nulos
    for i in ['CANT_OP_MON_DIG', 'CANT_OP_MON_OTR', 'CANT_OP_NOMON_DIG', 'CANT_OP_NOMON_OTR']:
        df_estudio[i] = df_estudio[i].fillna(0)
        df_estudio[i] = df_estudio[i].astype(int)
        
    # Eliminar los nulos de los montos
    for i in ['MONT_OP_MON_DIG', 'MONT_OP_MON_OTR']:
        df_estudio[i] = df_estudio[i].fillna(0)

    return df_estudio


## Archivo de reclamos

In [ ]:
def tratar_reclamos(periodo: str, df_cons: pd.DataFrame):

    file = dir + 'Gestar_{}.csv'.format(periodo)

    df_gestar = pd.read_csv(file, usecols=['ID_USER', 'STATEID', 'NIVEL 1'], index_col=False)

    # Me quedo solo con lo que son Reclamos y Quejas que se encuentren en estado CERRRADO o DERIVADO
    df_gestar = df_gestar[(((df_gestar['NIVEL 1']=='Reclamos') | (df_gestar['NIVEL 1']=='Queja')) & ((df_gestar['STATEID']==4) | (df_gestar['STATEID']==6)))]

    # Agrupo por estado del reclamo
    df_rec_agrup = df_gestar.groupby(['ID_USER', 'STATEID']) \
        .aggregate('count') \
        .sort_values(by='ID_USER') \
        .reset_index()
    df_rec_agrup.rename(columns={'NIVEL 1': 'CANT_RECL'}, inplace=True)

    # Pongo en columnas la cantidad de reclamos DERIVADOS
    df_rec_join = df_rec_agrup \
        .loc[(df_rec_agrup['STATEID']==4), ['ID_USER', 'CANT_RECL']] \
        .rename(columns = {'CANT_RECL': 'CANT_RECL_A'})
    df_estudio = pd.merge(df_estudio, df_rec_join, how='left', on = 'ID_USER', indicator=False)

    # Pongo en columnas la cantidad de reclamos CERRRADOS
    df_rec_join = df_rec_agrup \
        .loc[(df_rec_agrup['STATEID']==6), ['ID_USER', 'CANT_RECL']] \
        .rename(columns = {'CANT_RECL': 'CANT_RECL_C'})
    df_estudio = pd.merge(df_estudio, df_rec_join, how='left', on = 'ID_USER', indicator=False)

    # Convertir las columnas de cantidades a enteros previo sacar los valores nulos
    for i in ['CANT_RECL_A', 'CANT_RECL_A']:
        df_estudio[i] = df_estudio[i].fillna(0)
        df_estudio[i] = df_estudio[i].astype(int)

    return df_estudio


In [240]:
for periodo in ['202202', '202203', '202204']:
    
    df_parcial = tratar_crosseling(periodo, df_usuarios)
    df_parcial = tratar_movimientos(periodo, df_parcial)
    df_parcial = tratar_reclamos(periodo, df_parcial)
    
    #### Grabar ####
    file = dir + 'Crosseling_ampliado_{}.csv'.format(periodo)
    df_parcial.to_csv(file, index=False)
    print(file)

c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_ampliado_202202.csv
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_ampliado_202203.csv
c:\Users\e21719832\Documents\Python\trayecto3\Data\Export\Crosseling_ampliado_202204.csv


In [ ]:
#df_estudio[((df_estudio['MONT_OP_MON_OTR']!=0) & (df_estudio['MONT_OP_MON_DIG']!=0) & (df_estudio['CANT_OP_NOMON_DIG']!=0) & (df_estudio['CANT_OP_NOMON_OTR']!=0))]

In [ ]:
file = dir + 'Gestar_{}.csv'.format('202204')

df_gestar = pd.read_csv(file, usecols=['ID_USER', 'STATEID', 'NIVEL 1'], index_col=False)

# Me quedo solo con lo que son Reclamos y Quejas que se encuentren en estado CERRRADO o DERIVADO
df_gestar = df_gestar[(((df_gestar['NIVEL 1']=='Reclamos') | (df_gestar['NIVEL 1']=='Queja')) & ((df_gestar['STATEID']==4) | (df_gestar['STATEID']==6)))]

df_rec_agrup = df_gestar.groupby(['ID_USER', 'STATEID']) \
    .aggregate('count') \
    .sort_values(by='ID_USER') \
    .reset_index()
    
df_rec_agrup.rename(columns={'NIVEL 1': 'CANT_RECL'}, inplace=True)

In [ ]:
#df_rec_agrup.sort_values(by='CANT_RECL')